In [26]:
import polars as pl
from spatial_filtering import arrays, constants, direction_of_arrival
import importlib

importlib.reload(arrays)
importlib.reload(constants)
importlib.reload(direction_of_arrival)

meerkat_pos = pl.read_excel('meerkat positions.xlsx')
meerkat_pos = meerkat_pos.with_columns(distance_from_center_m=(pl.col('East')**2 + pl.col('North')**2 + pl.col('Up')**2).sqrt())
meerkat_pos_core = meerkat_pos.filter(pl.col('distance_from_center_m') < 750)
core_indices = [int(r[-2:]) for r in meerkat_pos_core['Antenna'].to_list()]
data_np = meerkat_pos_core[['East', 'North', 'Up']].to_numpy()

from spatial_filtering.arrays import Array
import numpy as np

array = arrays.Array(data_np)
N = array.num_antennas
f = 1.227574219e9              # Frequency 1.2276 GHz GPS L2 band.
wavelength = constants.c / f
snapshots = 2000


In [211]:
from dadanalyse import read_dada
GPS_CHANNEL = 50
POL = 1 
# J1644-4559_2024-02-16T11:21:22.092_29_73532270706688.dada
dada = read_dada("../../../data/gps.dada")
print(dada.get_header())
dada = dada.combined_data()



{'HEADER': 'DADA', 'HDR_VERSION': '1.0', 'HDR_SIZE': 4096, 'DADA_VERSION': '1.0', 'OBS_ID': 'None', 'FILE_SIZE': '3048214528', 'FILE_NUMBER': '0', 'UTC_START': '1708082482.092702505', 'MJD_START': '60356.47317236924195', 'OBS_OFFSET': '0', 'OBS_OVERLAP': '0', 'SOURCE': 'J1644-4559_Offset1', 'RA': '16:44:26.25', 'DEC': '-45:59:09.6', 'TELESCOPE': 'MeerKAT', 'INSTRUMENT': 'CBF-Feng', 'RECEIVER': 'L-band', 'FREQ': '1284000000.000000', 'BW': '856000000.000000', 'TSAMP': '4.7850467290', 'BYTES_PER_SECOND': '3424000000.000000', 'NBIT': 8, 'NDIM': 2, 'NPOL': 2, 'NCHAN': 64, 'NANT': 57, 'ORDER': 'TAFTP', 'INNER_T': 256, 'SYNC_TIME': '1708039531.000000', 'SAMPLE_CLOCK': '1712000000.000000', 'SAMPLE_CLOCK_START': '73532270706688', 'CHAN0_IDX': 1728, 'Lowest_freq (MHz)': 1217.125}


In [212]:
array.steering_vector([0,0], wavelength)

array([ 0.28936544-0.9572187j , -0.38367628+0.92346765j,
       -0.8083881 -0.58864988j,  0.89296643+0.45012327j,
        0.63408678+0.77326189j, -0.54450583-0.83875706j,
       -0.71451342+0.69962174j,  0.99988422-0.01521692j,
        0.12862974+0.99169269j, -0.99080042+0.13533121j,
        0.3375557 +0.94130556j, -0.91359294+0.40662998j,
        0.22982853+0.97323114j, -0.9492535 +0.31451198j,
        0.37860534-0.92555821j,  0.70924556+0.70496152j,
        0.77494962-0.63202301j, -0.48599853+0.87395963j,
        0.99237859-0.12322637j,  0.06867479-0.9976391j ,
       -0.94016331-0.34072417j,  0.07684229-0.99704326j,
        0.67667242-0.73628421j,  0.94554284+0.32549769j,
        0.53298596+0.84612409j,  0.01641594+0.99986525j,
        0.77013833+0.63787691j, -0.99937132+0.0354537j ,
       -0.11375228-0.99350914j, -0.98203582+0.1886946j ,
        0.32209399+0.94670769j, -0.86295217+0.50528561j,
        0.50018058-0.86592112j, -0.07006181+0.99754265j,
        0.65739175-0.753549j  ,

In [213]:
dada.shape

(208896, 57, 64, 2)

In [214]:
dada = dada[0:2000, core_indices, GPS_CHANNEL, POL].T
X = dada

In [6]:
from skyfield.api import load, Topos, EarthSatellite
from datetime import datetime
import numpy as np

# Location: MeerKAT Telescope (converted to decimal degrees)
lat = -30.71106
lon = 21.44389
elevation_m = 1086.6

# Time: Convert UNIX timestamp to UTC
timestamp = 1708082482.092702505
dt = datetime.utcfromtimestamp(timestamp)
dt

/var/folders/vv/d9ncb4ms2x1gl0mmkrvk8m7c0000gp/T/ipykernel_9853/1887256366.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(timestamp)


datetime.datetime(2024, 2, 16, 11, 21, 22, 92702)

In [230]:
array_factor = array.steering_vector([np.deg2rad(220.66994094517895), np.deg2rad(65.97484924086021)], wavelength)
#X = X.T
X_adj = X.copy()
for i, shift in enumerate(array_factor):
    X_adj[i, :] = X_adj[i, :] * array_factor[i].conj()

R = X @ X.conj().T / (N-1)
R_adj = X_adj @ X_adj.conj().T / (N-1)

In [231]:
X_adj

array([[ 22.959665 +8.0531855e+00j,  27.45773  -4.6990139e+01j,
         24.20182  +1.8419334e+01j, ...,   6.4681816-8.0101576e+00j,
         11.822832 -4.4634300e+01j, -31.098295 -1.9618767e+01j],
       [  6.3426657-8.9872465e+00j,  31.764166 -1.5525392e+01j,
         22.106506 +3.3618958e+00j, ...,  -7.689391 -2.9788032e+00j,
          8.985279 -1.4465986e+01j,  -5.9146366-3.8444988e+01j],
       [ -8.461606 -8.0871019e+00j,  30.669937 -2.2568893e+01j,
         34.34584  +6.3532047e+00j, ..., -27.007862 +2.4177994e+01j,
          3.3015137-5.9716831e+01j,  -3.8424134+3.7730439e+00j],
       ...,
       [-21.327545 -8.6680899e+00j,   6.315111 -3.7803696e+01j,
          8.90452  +1.1777500e+01j, ..., -39.88571  -1.7692107e+00j,
         12.110221 -3.8449219e+01j, -45.382633 -1.8585388e+01j],
       [ 19.063509 +4.5973717e+01j,   1.9999981-2.7620457e-03j,
         13.998606 -1.0193334e+00j, ...,  35.01654  +1.1951653e+01j,
         22.020695 +1.4969604e+01j, -29.97097  +2.1041410e+01j]

In [232]:
array_factor

array([ 0.98521064-0.1713476j , -0.81702237+0.57660597j,
        0.91551881-0.40227516j, -0.97013456+0.24256737j,
       -0.87409142-0.48576146j, -0.98936311-0.14546695j,
        0.73548116-0.67754518j,  0.92245159-0.38611276j,
        0.35741329+0.93394633j, -0.97816871+0.20781234j,
       -0.76481695-0.64424765j,  0.07071357+0.99749666j,
        0.52870965+0.84880275j,  0.57490302+0.81822156j,
        0.99564738+0.09320028j,  0.46626122+0.88464709j,
        0.92972319+0.36825914j, -0.15314914-0.98820309j,
       -0.00585464-0.99998286j, -0.9960426 +0.08887707j,
       -0.81955951+0.57299407j, -0.05844499+0.99829063j,
        0.24214976-0.97023888j,  0.00741343-0.99997252j,
        0.85806616-0.51353916j,  0.03638833+0.99933773j,
       -0.5335527 +0.84576682j, -0.54947241-0.83551186j,
       -0.74067205-0.67186673j,  0.61706576-0.78691159j,
       -0.99856655+0.05352428j, -0.98952302-0.14437517j,
        0.98706133+0.16034317j,  0.9053115 +0.42474826j,
       -0.60712774+0.79460425j,

In [233]:
import numpy as np

evals, evecs = np.linalg.eigh(R)
R

array([[44985.168    +0.j    , -1459.5714-5582.095j ,
         3569.9763+3302.238j , ...,  2061.7144+5717.2383j,
        -1153.7144-4108.262j ,  3663.786  -374.5j   ],
       [-1459.5714+5582.095j , 42051.       +0.j    ,
        -3400.6428+1791.9048j, ..., -2518.0952 -918.6905j,
         4717.905 +3536.2383j, -1269.5952+4113.1904j],
       [ 3569.9763-3302.238j , -3400.6428-1791.9048j,
        47091.336    +0.j    , ...,  7553.595 -2686.6428j,
        -3954.3096-2637.4287j, -1480.9286-4906.024j ],
       ...,
       [ 2061.7144-5717.2383j, -2518.0952 +918.6905j,
         7553.595 +2686.6428j, ..., 47223.836    +0.j    ,
        -5985.643  -355.6905j,  -861.5238-6947.2856j],
       [-1153.7144+4108.262j ,  4717.905 -3536.2383j,
        -3954.3096+2637.4287j, ..., -5985.643  +355.6905j,
        44721.383    +0.j    ,  1339.1906+5939.262j ],
       [ 3663.786  +374.5j   , -1269.5952-4113.1904j,
        -1480.9286+4906.024j , ...,  -861.5238+6947.2856j,
         1339.1906-5939.262j , 4171

In [234]:
R_adj

array([[44985.223  -2.6381115e-04j,  3708.6797 +4.4199419e+03j,
         4256.8506 +2.3513235e+03j, ...,  3448.6538 +5.0044272e+03j,
        -4245.786  +4.2684729e+02j, -1000.74445-3.5443022e+03j],
       [ 3708.6797 -4.4199414e+03j, 42050.99   +1.7336206e-04j,
         3689.4648 -1.0784878e+03j, ...,  2306.0237 +1.3664003e+03j,
        -5612.0605 +1.8078691e+03j, -3627.7468 -2.3172529e+03j],
       [ 4256.8506 -2.3513250e+03j,  3689.4648 +1.0784866e+03j,
        47091.32   -5.4255637e-05j, ...,  7511.088  -2.8032834e+03j,
        -4008.867  +2.5537388e+03j, -4995.8564 +1.1417706e+03j],
       ...,
       [ 3448.653  -5.0044277e+03j,  2306.025  -1.3664006e+03j,
         7511.0723 +2.8032834e+03j, ..., 47223.836  +1.1625744e-05j,
        -2823.1343 +5.2900215e+03j, -6995.0635 +2.7586703e+02j],
       [-4245.776  -4.2684763e+02j, -5612.058  -1.8078730e+03j,
        -4008.8699 -2.5537415e+03j, ..., -2823.1343 -5.2900220e+03j,
        44721.375  +0.0000000e+00j,  3278.695  +5.1301470e+03j]

In [235]:


theta_steps = 180 * 4
phi_steps = 360 * 4

output = direction_of_arrival.MUSICDOA2D().get_directions(
    array,
    R_adj, 
    num_interferers=5,
    wavelength = wavelength,
    theta_min_deg = 0,
    theta_max_deg = 90,
    theta_steps=theta_steps,
    phi_min_deg = 0,
    phi_max_deg = 360,
    phi_steps=phi_steps,
)

In [236]:

import plotly.express as px

import plotly.io as pio
pio.renderers.default = 'iframe'  

# Filter top 1% Q values (adjust as needed)
threshold = output['Q'].quantile(0.999975)
high_Q = output.filter(pl.col('Q') >= threshold)

# Create 3D scatter plot
fig = px.scatter_3d(
    high_Q,
    x='theta',  # azimuth
    y='phi',      # range
    z='Q',      # MUSIC power
    color='Q',
    color_continuous_scale='Jet',
    opacity=0.8
)

# Update layout for better visuals
fig.update_layout(
    title='High-Q MUSIC Spectrum Peaks',
    scene=dict(
        xaxis_title='Angle (degrees)',
        yaxis_title='Phi (degrees)',
        zaxis_title='Q (dB)'
    )
)

fig.show()

The MJD I want is 60356.47317236924195




In [159]:
from skyfield.api import load, EarthSatellite
from skyfield.iokit import parse_tle_file
import json
ts = load.timescale()

with load.open('gp2_2.tle') as f:
    data = json.load(f)

ts = load.timescale()
sats = [EarthSatellite.from_omm(ts, fields) for fields in data]

print('Loaded', len(data), 'satellites')

Loaded 54 satellites


In [200]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u

# Observer location (MeerKAT)
location = EarthLocation(lat=-30.71106*u.deg,
                         lon=21.44389*u.deg,
                         height=1086.6*u.m)

# Time of observation (UTC)
obstime = Time('2024-02-16 11:21:22')

# RA/DEC coordinates
ra_str = '16:44:26.25'
dec_str = '-45:59:09.6'

# Create SkyCoord object in ICRS (equatorial)
sky_coord = SkyCoord(ra=ra_str, dec=dec_str, unit=(u.hourangle, u.deg), frame='icrs')

# Create AltAz frame at observer location and time
altaz_frame = AltAz(obstime=obstime, location=location)

# Transform RA/DEC to AltAz
altaz = sky_coord.transform_to(altaz_frame)

# Output altitude and azimuth
print("Source RA/DEC")
print(f"Altitude: {altaz.alt:.4f}")
print(f"Azimuth: {altaz.az:.4f}")

print("Source Theta / Phi")
print(f"Theta: {90 - altaz.alt.degree}")
print(f"Phi: {(90 - altaz.az.degree) % 360}")

Source RA/DEC
Altitude: 24.0252 deg
Azimuth: 229.3301 deg
Source Theta / Phi
Theta: 65.97484924086021
Phi: 220.66994094517895


In [208]:
from skyfield.api import wgs84, Topos

meerkat = Topos(latitude_degrees=-30.71106,
                longitude_degrees=21.44389,
                elevation_m= 1086.6)

obs_site = meerkat
t_obs = ts.utc(2024, 2, 16, 11, 21, 22)
sats_visible = []
sats_processed = set()

for satellite in sats:
    # There are some duplicate TLEs - don't worry about this.
    # any of them should give essentially the same result.
    if satellite.name in sats_processed:
        continue
    event_dict = {}
    event_dict['name'] = satellite.name

    topocentric = (satellite - obs_site).at(t_obs)
    alt, az, _ = topocentric.altaz()
    event_dict['alt'] = alt.degrees
    event_dict['az'] = az.degrees
    if alt.degrees > 0:
        sats_visible.append(event_dict)
    sats_processed.add(satellite.name)

df = pl.from_records(sats_visible)
#df.write_excel('sats.xlsx')
#len(sats_processed)
print(len(sats_visible))
df.with_columns(theta=pl.lit(90) - pl.col('alt'), phi=(pl.lit(90) - pl.col('az'))).sort(by="alt", descending=False)


10


name,alt,az,theta,phi
str,f64,f64,f64,f64
"""NAVSTAR 69 (USA 248)""",4.096493,141.102433,85.903507,-51.102433
"""NAVSTAR 81 (USA 319)""",8.048966,45.01476,81.951034,44.98524
"""NAVSTAR 47 (USA 150)""",16.642376,91.633704,73.357624,-1.633704
"""NAVSTAR 79 (USA 304)""",22.556092,245.454394,67.443908,-155.454394
"""NAVSTAR 67 (USA 239)""",36.857815,20.403034,53.142185,69.596966
"""NAVSTAR 61 (USA 199)""",37.980936,309.60797,52.019064,-219.60797
"""NAVSTAR 64 (USA 206)""",44.369909,123.403165,45.630091,-33.403165
"""NAVSTAR 78 (USA 293)""",45.372665,219.380969,44.627335,-129.380969
"""NAVSTAR 43 (USA 132)""",49.903986,140.065598,40.096014,-50.065598


In [209]:
output.with_columns(dist_from_theta=90 - pl.col('theta'), dist_from_phi = (90-pl.col('phi')) % 360).sort(by="Q", descending=True).head(10)

phi,theta,Q,dist_from_theta,dist_from_phi
f64,f64,f64,f64,f64
262.182071,77.732962,0.0,12.267038,187.817929
189.881862,5.006954,-0.02927,84.993046,260.118138
31.772064,12.01669,-0.123808,77.98331,58.227936
299.207783,22.906815,-0.225441,67.093185,150.792217
331.230021,74.353268,-0.304817,15.646732,118.769979
197.136901,12.392211,-0.347596,77.607789,252.863099
265.184156,67.969402,-0.404166,22.030598,184.815844
13.259208,5.757997,-0.410941,84.242003,76.740792
70.799166,56.328234,-0.411669,33.671766,19.200834


In [163]:
from astropy.time import Time

# Your MJD value (example)
mjd = 60356.47317236924195

# Create Time object
t = Time(mjd, format='mjd', scale='utc')

# Print UTC as ISO format
print("UTC Time:", t.utc.iso)

# Or get components
print("Year:", t.utc.datetime.year)
print("Datetime:", t.utc.datetime)

UTC Time: 2024-02-16 11:21:22.093
Year: 2024
Datetime: 2024-02-16 11:21:22.092702


In [164]:
## Near-field limit calculation
1500**2 / (constants.c / 1.276e9)

9576625.173138944

In [83]:
(90 - 229.33) % 360

220.67

In [85]:
90 - 24.03

65.97

In [88]:
np.sin(np.deg2rad(65.97)) * np.sin(np.deg2rad(220.67))

np.float64(-0.5952199413798546)